In [1]:
import numpy as np
import pandas as pd
from scipy.stats import uniform
import cbadc as cb
from itertools import product
from numpy.linalg import LinAlgError
from scipy.stats import qmc

In [2]:
def generate_dataset(form, order, implementation, num_samples, min_snr, parameter_ranges, discrete_params):
    """
    Generate a dataset based on the selected form, order, implementation
    
    Args:
    - form (str): The (continuous) form of the filter ('FB', 'FF')
    - order (int): The order of the filter (2, 3, 4)
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')
    - num_samples (int): Initial number of samples to run LHS
    - min_snr (float): Minimum SNR accepted.
    - parameter_ranges (list): List with the range of each parameter
    - discrete_parameters (list): List with the discrete parameters and each values
    
    Returns:
    - dataset (pd.DataFrame): A DataFrame containing the generated dataset,
                              named: dataset_form_order_implementation.csv
    """
    
    counter = 0 # counter to reach at least num_samples samples in the final dataset
    
    # initialize lists to store the parameters and corresponding SNR values after simulation:
    snr_values = []
    all_params = []
    
    while counter <= num_samples:
        # Latin Hypercube Sampling (LHS) to sample the parameter space:
        lhs_samples = lhs_sampling(parameter_ranges, num_samples, discrete_params)

        # iterate through each set of parameters from the LHS samples:
        for sample in lhs_samples:
            # map the sampled values to parameter names:
            if implementation == 'Active-RC':
                params = {'Bw': sample[0], 'osr': int(sample[1]), 'nlev': int(sample[2]), 'Hinf': sample[3], 
                          'Cint': sample[4], 'gm': sample[5], 'Ro': sample[6], 'Co': sample[7]}
            elif implementation == 'Gm-C':
                params = {'Bw': sample[0], 'osr': int(sample[1]), 'nlev': int(sample[2]), 'Hinf': sample[3], 
                          'Cint': sample[4], 'Ro': sample[5], 'Cp': sample[6], 'v_n': sample[7], 
                          'slew_rate': sample[8], 'output_swing': sample[9]}

            # apply parameter relationships filters to ensure validity (not implemented yet):
            if not filter_parameter_relationships(params, implementation):
                continue # skip this sample if it doesn't pass the filter

            # simulate the circuit and get the SNR value:
            snr = simulate_circuit(form, order, implementation, params)

            # store the the parameters and corresponding SNR value:
            if snr is not None and np.isfinite(snr) and snr > min_snr:
                snr_values.append(snr)
                all_params.append(params)
                counter += 1
    
    # create a DataFrame and save it as CSV:
    dataset = pd.DataFrame(all_params)
    dataset['SNR'] = snr_values
    filename = f"datasets/dataset_{form}_{order}_{implementation}.csv"
    dataset.to_csv(filename, index=False)
    print(f"\n\tDataset saved as {filename}\n")
    
    return dataset

In [3]:
def lhs_sampling(parameter_ranges, num_samples, discrete_params):
    """
    Latin Hypercube Sampling (LHS) for a given set of parameter ranges,
    handling both continuous and discrete parameters

    Args:
    - parameter_ranges (list): List of tuples [(min, max), ...] defining the range for each parameter
    - num_samples (int): Number of samples to generate
    - discrete_params (list): List of tuples [(index, values)] where:
        - index: The position of the discrete parameter in parameter_ranges
        - values: A list of possible discrete values

    Returns:
    - samples (list): A NumPy array of shape (num_samples, len(parameter_ranges)) containing the generated samples
    """

    # separate continuous and discrete parameters:
    continuous_params = [p for i, p in enumerate(parameter_ranges) if i not in [d[0] for d in discrete_params]]
    discrete_indices = {d[0]: d[1] for d in discrete_params}

    # create an LHS design only for continuous parameters:
    sampler = qmc.LatinHypercube(d=len(continuous_params)) # d = number of continuous parameters
    sample_cont = sampler.random(num_samples) # generate LHS samples in [0,1] range
    
    # rescale LHS samples to match the specified ranges for continuous parameters:
    cont_scaled = qmc.scale(sample_cont, 
                            [p[0] for p in continuous_params], # min values
                            [p[1] for p in continuous_params]) # max values
    
    # initialize the result matrix:
    samples = np.zeros((num_samples, len(parameter_ranges)))

    # insert continuous values into the result matrix:
    cont_idx = 0 # index to track continuous parameters in cont_scaled
    for i, p in enumerate(parameter_ranges):
        if i in discrete_indices: # skip discrete parameters
            continue
        samples[:, i] = cont_scaled[:, cont_idx] # assign LHS-sampled values
        cont_idx += 1

    # insert discrete parameters using random selection with replacement:
    for idx, values in discrete_indices.items():
        samples[:, idx] = np.random.choice(values, size=num_samples, replace=True)

    return samples

In [4]:
# LHS test:

parameter_ranges = [
            (1e3, 1e9), # Bw (Bandwidth)
            [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
            [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
            (1e-3, 1e1), # Hinf (Infinity Gain)
            (1e-12, 1e-6), # Cint (Integration Capacitance)
            (1e-3, 1e-1), # gm (Transconductance)
            (1e3, 1e6), # Ro (Output Resistance)
            (1e-12, 1e-6), # Co (Output Capacitance)
        ]
discrete_params = [
        (1, [8, 16, 32, 64, 128, 256]), # osr (discrete)
        (2, [2, 3, 4, 5]) # nlev (discrete)
    ]

num_samples = 100 # number of samples
lhs_samples = lhs_sampling(parameter_ranges, num_samples, discrete_params)

print(lhs_samples)
print(len(lhs_samples))

[[4.65585572e+08 8.00000000e+00 4.00000000e+00 2.46235264e+00
  8.21944392e-07 2.68998293e-02 6.33190785e+05 1.19348703e-07]
 [6.60794421e+08 3.20000000e+01 4.00000000e+00 4.56545925e+00
  6.14523971e-07 2.65172997e-02 2.17673443e+05 6.55233708e-08]
 [2.33158198e+08 2.56000000e+02 5.00000000e+00 6.47528835e+00
  4.95644121e-07 5.64738358e-02 1.46722705e+05 8.11430447e-07]
 [9.67999471e+08 8.00000000e+00 5.00000000e+00 2.74890756e+00
  8.91527637e-07 5.88190356e-02 9.72142208e+05 2.10123470e-07]
 [8.56819184e+08 3.20000000e+01 4.00000000e+00 6.06030615e+00
  4.17525920e-07 3.92507025e-02 2.96552523e+05 7.12430988e-07]
 [4.25328459e+08 1.60000000e+01 3.00000000e+00 1.85495337e+00
  1.64478249e-07 7.46182485e-02 3.89199469e+05 4.57023213e-07]
 [9.76500447e+08 8.00000000e+00 4.00000000e+00 7.76288056e+00
  1.31547374e-08 7.56283859e-02 9.25045148e+05 5.43123058e-07]
 [7.50473103e+08 2.56000000e+02 3.00000000e+00 2.68133460e+00
  4.39564846e-07 9.91953590e-02 2.01409800e+05 5.05785713e-07]


In [5]:
def simulate_circuit(form, order, implementation, params):
    """
    Simulate the circuit and return the SNR value based on the selected
    form, order and implementation
    
    Args:
    - form (str): The (continuous) form of the filter ('FB', 'FF')
    - order (int): The order of the filter (2, 3, 4)
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')
    - params (dict): A dictionary containing the parameters for the simulation
    
    Returns:
    - max_snr (float): The maximum SNR value obtained from the simulation
    """
    
    # define parameters:
    Bw = params.get('Bw')
    osr = params.get('osr')
    nlev = params.get('nlev')
    Hinf = params.get('Hinf')
    f0 = 0.
    tdac = [0, 1]
    
    # compute the sampling frequency:
    fs = Bw * osr * 2
    
    # synthesize the optimal NTF (Noise Transfer Function)
    ntf = cb.delsig.synthesizeNTF(order, osr, 2, Hinf, f0)
    
    # create analog frontend:
    dt = 1.0 / fs
    ABCDc, tdac2 = cb.delsig.realizeNTF_ct(ntf, form, tdac)
    analog_frontend = cb.AnalogFrontend.ctsdm(ABCDc, tdac2, dt, nlev)
    analog_frontend.dt = 1.0 / fs
    
    # run the SNR simulation (depending on the implementation):
    if implementation == 'Active-RC':
        max_snr = simulate_snr_active_rc(params, order, osr, analog_frontend)
    elif implementation == 'Gm-C':
        max_snr = simulate_snr_gm_c(params, order, osr, analog_frontend)    
    else:
        raise ValueError(f"Unknown implementation type: {implementation}")
    
    return max_snr



def simulate_snr_active_rc(params, order, osr, analog_frontend):
    """
    Simulate the SNR for Active-RC implementation
    
    Args:
    - params (dict): The parameter set
    - order (int): The order of the filter (2, 3, 4)
    - osr (int): The oversampling ratio
    
    Returns:
    - snr (float): The simulated SNR value.
    """
    
    # define parameters:
    Cint = np.ones(order) * params.get('Cint')
    gm = np.ones(order) * params.get('gm')
    Ro = np.ones(order) * params.get('Ro')
    Co = np.ones(order) * params.get('Co')
    
    # simulate SNR:
    try:
        ActiveRC_analog_frontend = cb.ActiveRC(
            analog_frontend, Cint, gm, Ro, Co
        )
                
        snr_ActiveRC, amp_ActiveRC, _ = ActiveRC_analog_frontend.simulateSNR(osr)
        max_snr_ActiveRC = max(snr_ActiveRC)
        
        if not np.isnan(max_snr_ActiveRC):
            print(f"Achieved SNR: {max_snr_ActiveRC} db")
            return max_snr_ActiveRC
        else:
            return None
        
    except (LinAlgError, ValueError) as e:
        print(f"Error with Active-RC parameters {params}: {e}")
        return None



# Function to simulate SNR for Gm-C
def simulate_snr_gm_c(params, order, osr, analog_frontend):
    """
    Simulate the SNR for Gm-C implementation
    
    Args:
    - params (dict): The parameter set
    - order (int): The order of the filter (2, 3, 4)
    - osr (int): The oversampling ratio
    
    Returns:
    - snr (float): The simulated SNR value
    """ 
    
    # define parameters:
    Cint = np.ones(order) * params.get('Cint')
    Ro = np.ones(order) * params.get('Ro')
    Cp = np.ones(order) * params.get('Cp')
    v_n = np.ones(order) * params.get('v_n')
    slew_rate = np.ones(order) * params.get('slew_rate')
    v_out_max = np.ones(order) * params.get('output_swing')
    v_out_min = -v_out_max
    
    # simulate SNR:
    try:
        GmC_analog_frontend = cb.GmC(
            analog_frontend, Cint, Ro, Cp, v_n, v_out_max, v_out_min, slew_rate
        )
    
        snr_GmC, amp_GmC, _ = GmC_analog_frontend.simulateSNR(osr)
        max_snr_GmC = max(snr_GmC)

        if not np.isnan(max_snr_GmC):
            print(f"Achieved SNR: {max_snr_GmC} db")
            return max_snr_GmC
        else:
            return None
    
    except (LinAlgError, ValueError) as e:
        print(f"Error with Gm-C parameters {params}: {e}")
        return None

In [6]:
# simulation test:

params = {'form': 'FB', 'order': 2, 'Bw': 20e3, 'osr': 64, 'nlev': 2.0, 'Hinf': 1.5, 'implementation': 'Active-RC', 'Cint': 1e-12, 'gm': 0.01, 'Ro': 1e4, 'Co': 1e-12}
snr = simulate_circuit('FB', 2, 'Active-RC', params)

/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_ltisys.py:599: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  self.num, self.den = normalize(*system)
/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_filter_design.py:1125: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  b, a = normalize(b, a)
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 73.90913681655965 db


In [7]:
def filter_parameter_relationships(params, implementation):
    """
    Apply relationships-based filters to ensure that the parameters 
    comply with physical and stability constraints

    Args:
    - params (dict): The parameters set
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')

    Returns:
    - cond (bool): True if the parameters are valid, False otherwise
    """
    
    # add relationships between parameters as needed:
    
    """
    Example:
    
    if implementation == 'Active-RC':
        gm = params.get('gm')
        Ro = params.get('Ro')

        if gm * Ro > 1e3:
            return False # invalid relationship

    elif implementation == 'Gm-C':
        gm = params.get('gm')
        Ro = params.get('Ro')

        if gm * Ro < 1e-2:
            return False # invalid relationship
    """
    
    return True # if no issues, return True to indicate the parameters are valid

In [12]:
# dataset generator test:

form = 'FB'
order = 2
implementation = 'Active-RC'
num_samples = 50
min_snr = 50

# define parameter ranges (set as function args better?):
if implementation == 'Active-RC':
    parameter_ranges = [
        (10e3, 10e9), # Bw (Bandwidth)
        [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
        [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
        (1.0, 2.0), # Hinf (Infinity Gain)
        (100e-15, 5e-12), # Cint (Integration Capacitance)
        (1e-6, 1e-3), # gm (Transconductance)
        (100e3, 10e6), # Ro (Output Resistance)
        (50e-15, 1e-12), # Co (Output Capacitance)
    ]
elif implementation == 'Gm-C':
    parameter_ranges = [
        (10e3, 10e9), # Bw (Bandwidth)
        [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
        [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
        (1.0, 2.0), # Hinf (Infinity Gain)
        (100e-15, 2e-12), # Cint (Integration Capacitance)
        (10e3, 1e6), # Ro (Output Resistance)
        (1e-15, 100e-15), # Cp (Capacitor)
        (10e-9, 1e-6), # v_n (Noise Voltage)
        (1e-6, 100e-6), # slew_rate (Slew Rate)
        (0.5, 3.0), # output_swing (Output Swing)
    ]

# define discrete parameters:
discrete_params = [
    (1, [8, 16, 32, 64, 128, 256]), # osr (discrete)
    (2, [2, 3, 4, 5]) # nlev (discrete)
]

dataset = generate_dataset(form, order, implementation, num_samples, min_snr, parameter_ranges, discrete_params)
print(dataset.head())

INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Error with Active-RC parameters {'Bw': np.float64(2521464022.1312065), 'osr': 8, 'nlev': 4, 'Hinf': np.float64(1.4023671221564067), 'Cint': np.float64(5.132323333063712e-13), 'gm': np.float64(0.0006048753109920111), 'Ro': np.float64(8984771.656387355), 'Co': np.float64(6.891072386394927e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis
Error with Active-RC parameters {'Bw': np.float64(4898858717.672142), 'osr': 64, 'nlev': 3, 'Hinf': np.float64(1.3372739360192283), 'Cint': np.float64(1.0937773562740378e-12), 'gm': np.float64(0.00015843514947398745), 'Ro': np.float64(8152500.176228689), 'Co': np.float64(4.2135881308653573e-13)}: Failed to find a finite solution.


INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 3.9206693925404252 db
Error with Active-RC parameters {'Bw': np.float64(6575986715.712154), 'osr': 16, 'nlev': 4, 'Hinf': np.float64(1.9381648061681007), 'Cint': np.float64(3.204072851603955e-12), 'gm': np.float64(8.674468810535993e-05), 'Ro': np.float64(4477328.305800382), 'Co': np.float64(9.46775271775976e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


...

Achieved SNR: 32.74108514205578 db
Error with Active-RC parameters {'Bw': np.float64(4420682871.925284), 'osr': 32, 'nlev': 5, 'Hinf': np.float64(1.725618965110442), 'Cint': np.float64(6.554257287019424e-13), 'gm': np.float64(3.463692642984931e-05), 'Ro': np.float64(6822000.91853522), 'Co': np.float64(4.955367585831753e-13)}: Failed to find a finite solution.


INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 43.62918455486273 db
Error with Active-RC parameters {'Bw': np.float64(2645950379.6695232), 'osr': 64, 'nlev': 3, 'Hinf': np.float64(1.5049782095787543), 'Cint': np.float64(1.0340630826083363e-12), 'gm': np.float64(0.0005322004760942514), 'Ro': np.float64(915148.7791329835), 'Co': np.float64(8.972498832882435e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 2.9718226645027923 db
Error with Active-RC parameters {'Bw': np.float64(9023018643.381052), 'osr': 32, 'nlev': 2, 'Hinf': np.float64(1.5794360598539106), 'Cint': np.float64(2.3905088534222068e-12), 'gm': np.float64(0.00021347304258256317), 'Ro': np.float64(4807495.14807394), 'Co': np.float64(3.9758342859176803e-13)}: Failed to find a finite solution.

	Dataset saved as datasets/dataset_FB_2_Active-RC.csv

             Bw  osr  nlev      Hinf          Cint        gm            Ro  \
0  5.707472e+09  256     3  1.729690  3.345706e-12  0.000962  2.726413e+06   
1  9.463414e+07  128     3  1.351404  1.734872e-12  0.000712  2.381135e+06   
2  5.801101e+08   64     3  1.873414  2.891152e-12  0.000737  9.212645e+06   
3  3.024834e+09  256     2  1.409903  3.249928e-12  0.000337  7.368698e+06   
4  3.908940e+09  128     3  1.834400  1.506762e-12  0.000453  9.591841e+06   

             Co        SNR  
0  4.520814e-13  74.891326  
1  1.469849e-13  84.143820  
2  9.397599e-13  58.3